In [3]:
import requests
from datetime import datetime, timezone
from dateutil import parser

# Mapeo de códigos a descripción
TIPOS_PROBLEMA = {
    1: "Consultar Nota Informativa",
    2: "Suministro sólo a hospitales.",
    3: "El médico prescriptor deberá determinar la posibilidad de utilizar otros tratamientos comercializados.",
    4: "Desabastecimiento temporal.",
    5: "Existe/n otro/s medicamento/s con el mismo principio activo y para la misma vía de administración.",
    6: "Existe/n otro/s medicamento/s con los mismos principios activos y para la misma vía de administración.",
    7: "Se puede solicitar como medicamento extranjero.",
    8: "Se recomienda restringir su prescripción reservándolo para casos en que no exista una alternativa apropiada.",
    9: "El titular de autorización de comercialización está realizando una distribución controlada al existir unidades limitadas."
}

def _parse_fecha(valor):
    """
    Detecta si 'valor' es:
     - int/float o str de dígitos (milisegundos UNIX): lo convierte a ISO8601 UTC
     - cualquier otra str: lo intenta parsear con dateutil
    Si falla, devuelve el valor original.
    """
    # Timestamp UNIX en ms
    if isinstance(valor, (int, float)) or (isinstance(valor, str) and valor.isdigit()):
        ts = int(valor) / 1000
        return datetime.fromtimestamp(ts, tz=timezone.utc).isoformat()
    # Cualquier otra cadena
    if isinstance(valor, str):
        try:
            dt = parser.parse(valor)
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=timezone.utc)
            return dt.isoformat()
        except parser.ParserError:
            return valor
    # Otros tipos (None, bool, etc.)
    return valor

def obtener_problema_suministro(cn: str) -> dict | list:
    """
    Llama a la API CIMA y post-procesa:
     - Mapea tipoProblemaSuministro → descripcion
     - Renombra fini → fecha_inicio, ffin → fecha_fin y convierte fechas
    Devuelve un dict (si solo hay un registro) o bien una lista de dicts.
    """
    base_url = "https://cima.aemps.es/cima/rest/psuministro/v2"
    url = f"{base_url}/cn/{cn}"
    resp = requests.get(url, headers={"Accept": "application/json"})
    resp.raise_for_status()
    raw = resp.json()

    # Determinar lista de items a procesar
    if isinstance(raw, dict) and "resultados" in raw and isinstance(raw["resultados"], list):
        items = raw["resultados"]
        devolver_como_lista = False  # devolvemos el dict completo con key "resultados"
    else:
        # puede venir un dict único o directamente una lista
        if isinstance(raw, list):
            items = raw
            devolver_como_lista = True
        else:
            items = [raw]
            devolver_como_lista = True  # devolvemos lista con un solo elemento

    # Post-procesado
    for item in items:
        # 1. descripción del tipo de problema
        codigo = item.get("tipoProblemaSuministro")
        item["tipoProblemaSuministro_descripcion"] = TIPOS_PROBLEMA.get(codigo, "Desconocido")
        # 2. fechas
        if "fini" in item:
            item["fecha_inicio"] = _parse_fecha(item.pop("fini"))
        if "ffin" in item:
            item["fecha_fin"]   = _parse_fecha(item.pop("ffin"))

    # Devolver en la forma original esperada
    if devolver_como_lista:
        return items
    else:
        raw["resultados"] = items
        return raw

if __name__ == "__main__":
    cn = "719098"
    datos = obtener_problema_suministro(cn)
    print(datos)


[{'cn': '719098', 'nombre': 'ATOMOXETINA CINFA 10 MG CAPSULAS DURAS EFG, 7 cápsulas (Blister PVC/PE/PCTFE-Al)', 'tipoProblemaSuministro': 5, 'activo': True, 'observ': 'Existe/n otro/s medicamento/s con el mismo principio activo y para la misma vía de administración.', 'tipoProblemaSuministro_descripcion': 'Existe/n otro/s medicamento/s con el mismo principio activo y para la misma vía de administración.', 'fecha_inicio': '2025-05-11T22:00:00+00:00', 'fecha_fin': '2025-06-29T22:00:00+00:00'}]
